# Chapter 44: Eventing and Asynchronous Work

Synchronous request-response cycles cannot handle long-running operations, external API calls, or distributed transaction coordination without destroying user experience. PostgreSQL provides mechanisms for asynchronous processing—from lightweight pub/sub notifications to durable job queues—allowing applications to defer work, communicate between services, and coordinate distributed systems. This chapter covers the architectural patterns, reliability guarantees, and operational constraints of event-driven architectures built on PostgreSQL.

## 44.1 The Landscape of Asynchronous Patterns

PostgreSQL supports three distinct asynchronous communication paradigms, each with different durability and delivery guarantees:

1. **LISTEN/NOTIFY**: Ephemeral pub/sub for real-time signaling (at-most-once delivery)
2. **Outbox Pattern**: Transactional message publishing with guaranteed delivery (at-least-once)
3. **Job Queues**: Durable work distribution with competitive consumption (exactly-once processing)

Understanding the trade-offs between latency, durability, and complexity determines pattern selection.

## 44.2 LISTEN/NOTIFY (Pub/Sub)

PostgreSQL's notification system provides real-time, connection-oriented messaging between database sessions.

### 44.2.1 Mechanism and Payload Limits

The NOTIFY command sends a message to all connected sessions listening on a specific channel:

```sql
-- Session A: Subscribe to channel
LISTEN user_updates;

-- Session B: Publish message
NOTIFY user_updates, '{"user_id": 123, "action": "profile_updated"}';

-- Session A receives: Asynchronous notification "user_updates" 
-- with payload "{"user_id": 123, "action": "profile_updated"}" from server process PID 12345
```

**Critical Constraints:**
- **Payload limit**: 8000 bytes (uncompressed). Larger payloads truncate silently in older versions, error in PostgreSQL 14+.
- **Connection-bound**: Notifications queue per session; if disconnected, messages are lost permanently.
- **No persistence**: Notifications exist only in server memory; crash or restart clears all undelivered messages.
- **Single delivery**: Each notification delivers to each connected listener exactly once (if connected).

**Payload Size Management:**
```sql
-- Anti-pattern: Large JSON payloads
NOTIFY order_events, (SELECT row_to_json(orders)::text FROM orders WHERE order_id = 123);
-- Risk: Exceeds 8KB, truncates or fails

-- Pattern: Reference pattern (durable, small payload)
NOTIFY order_events, json_build_object('order_id', 123, 'event_type', 'created')::text;
-- Client receives notification, queries database for full details if needed
```

### 44.2.2 Client Implementation Patterns

Database drivers handle notifications via separate connections or async callbacks:

**Node.js (pg driver):**
```javascript
const { Client } = require('pg');

const client = new Client();
await client.connect();

// Dedicated connection for notifications (cannot be used for queries)
client.on('notification', (msg) => {
    console.log('Channel:', msg.channel);
    console.log('Payload:', msg.payload);
    console.log('PID:', msg.processId);
    
    // Acknowledge by processing (no explicit ACK needed)
    processEvent(JSON.parse(msg.payload));
});

await client.query('LISTEN user_updates');

// Keep connection alive (heartbeats or periodic queries)
setInterval(async () => {
    await client.query('SELECT 1');
}, 30000);
```

**Python (asyncpg):**
```python
import asyncpg
import asyncio

async def listener():
    conn = await asyncpg.connect(dsn='postgresql://...')
    await conn.add_listener('inventory_changes', handle_notification)
    
    # Keep running
    while True:
        await asyncio.sleep(3600)

def handle_notification(connection, pid, channel, payload):
    print(f'Received on {channel}: {payload}')
    # Process asynchronously

# Critical: Use dedicated connection; cannot share with transaction pool
```

**Go (pgx):**
```go
conn, _ := pgx.Connect(context.Background(), os.Getenv("DATABASE_URL"))
defer conn.Close(context.Background())

_, err := conn.Exec(context.Background(), "LISTEN inventory_updates")
if err != nil { log.Fatal(err) }

for {
    notification, err := conn.WaitForNotification(context.Background())
    if err != nil { log.Fatal(err) }
    
    fmt.Printf("Notification: %s\n", notification.Payload)
}
```

### 44.2.3 Reliability Limitations and Mitigations

**The Missed Notification Problem:**
If a listener disconnects (network blip, application restart) between NOTIFY and LISTEN reconnection, messages are lost forever.

**Mitigation Strategies:**

1. **Complementary Polling Pattern** (Eventual Consistency):
```sql
-- Fallback polling for missed notifications
SELECT MAX(event_id) as last_processed FROM processed_events;

-- Poll every 30 seconds for events > last_processed
SELECT * FROM events 
WHERE event_id > $1 
  AND created_at > now() - interval '5 minutes'
ORDER BY event_id;
```

2. **Logical Decoding** (Reliable Streaming):
For guaranteed delivery, use logical replication slots instead of NOTIFY:
```sql
-- Create replication slot (pg_recvlogical or driver equivalent)
SELECT pg_create_logical_replication_slot('events_slot', 'pgoutput');

-- Stream changes via protocol (not SQL) for guaranteed delivery
-- Requires separate logical replication connection
```

3. **Application-Level Deduplication:**
```sql
-- Track processed notification IDs (for idempotent processing)
CREATE TABLE notification_log (
    notification_id uuid PRIMARY KEY,
    channel text,
    processed_at timestamptz DEFAULT now()
);

-- On receive, insert with ON CONFLICT DO NOTHING
INSERT INTO notification_log (notification_id, channel) 
VALUES ($1, 'user_updates')
ON CONFLICT DO NOTHING;

-- If rows affected = 0, duplicate notification (ignore)
```

### 44.2.4 Use Cases and Anti-Patterns

**Appropriate Uses:**
- **Cache invalidation**: Notify edge caches to purge keys when database updates occur
- **Real-time UI updates**: WebSocket broadcasts when underlying data changes
- **Configuration reload**: Signal applications to refresh in-memory settings
- **Monitoring alerts**: Low-frequency critical events (error rate spikes)

**Anti-Patterns:**
- **High-frequency events** (>100/sec): Exhausts network buffers, causes latency
- **Critical financial events**: No durability guarantee; use Outbox instead
- **Distributed locking**: No fencing tokens; use advisory locks instead

## 44.3 The Outbox Pattern (Reliable Messaging)

Microservices architectures face the "dual write problem": updating the database and sending a message to a broker cannot both be atomic in separate systems.

### 44.3.1 The Problem Statement

```go
// Anti-pattern: Non-atomic dual write
func CreateOrder(ctx context.Context, order Order) error {
    tx, _ := db.Begin(ctx)
    defer tx.Rollback(ctx)
    
    // Write to database
    err := tx.Exec(ctx, "INSERT INTO orders ...", order)
    if err != nil { return err }
    
    tx.Commit(ctx)  // Database committed
    
    // Send to Kafka (can fail independently)
    err = kafkaProducer.Send(order)  // If this fails, order exists but no event emitted
    // Or worse: if process crashes here, inconsistency
    
    return err
}
```

### 44.3.2 Outbox Table Implementation

The Outbox Pattern ensures atomicity by writing events to a PostgreSQL table (the outbox) in the same transaction as business data changes:

```sql
-- Outbox table design
CREATE TABLE outbox_events (
    event_id bigserial PRIMARY KEY,
    aggregate_type text NOT NULL,      -- 'order', 'user', etc.
    aggregate_id text NOT NULL,        -- 'order-123'
    event_type text NOT NULL,          -- 'OrderCreated', 'PaymentReceived'
    payload jsonb NOT NULL,            -- Event data
    metadata jsonb,                    -- Correlation IDs, timestamps
    created_at timestamptz DEFAULT now(),
    published_at timestamptz,         -- NULL until relay confirms
    retry_count int DEFAULT 0,
    
    CONSTRAINT valid_event CHECK (payload <> '{}')
);

-- Index for efficient polling
CREATE INDEX idx_outbox_unpublished 
ON outbox_events (created_at) 
WHERE published_at IS NULL;

-- Partition by time if high volume (monthly partitions)
-- CREATE TABLE outbox_events_2024_01 PARTITION OF outbox_events ...
```

**Transactional Event Publishing:**
```sql
-- Within application transaction
BEGIN;

-- 1. Business logic
INSERT INTO orders (order_id, customer_id, total) 
VALUES ('ord-456', 'cust-789', 10000);

-- 2. Atomic event write (same transaction)
INSERT INTO outbox_events (
    aggregate_type, 
    aggregate_id, 
    event_type, 
    payload,
    metadata
) VALUES (
    'order',
    'ord-456',
    'OrderCreated',
    jsonb_build_object(
        'order_id', 'ord-456',
        'customer_id', 'cust-789',
        'total_cents', 10000,
        'line_items', (SELECT jsonb_agg(item) FROM order_items WHERE order_id = 'ord-456')
    ),
    jsonb_build_object(
        'correlation_id', current_setting('app.correlation_id', true),
        'timestamp', extract(epoch from now())
    )
);

COMMIT;
-- Both order and event committed atomically
-- If commit fails, neither exists (consistency maintained)
```

### 44.3.3 The Relay Process (Message Delivery)

A separate process polls the outbox and publishes to the message broker:

```python
# Relay implementation (Python example)
import asyncio
import asyncpg

async def relay_loop():
    conn = await asyncpg.connect(dsn='postgresql://...')
    
    while True:
        # Fetch batch of unpublished events
        rows = await conn.fetch("""
            SELECT event_id, aggregate_type, event_type, payload, metadata
            FROM outbox_events
            WHERE published_at IS NULL
            ORDER BY event_id
            LIMIT 100
            FOR UPDATE SKIP LOCKED
        """)
        
        for row in rows:
            try:
                # Publish to Kafka/RabbitMQ/SNS
                await message_bus.publish(
                    topic=f"{row['aggregate_type']}-{row['event_type']}",
                    message=row['payload'],
                    headers=row['metadata']
                )
                
                # Mark as published (or delete)
                await conn.execute("""
                    UPDATE outbox_events 
                    SET published_at = now()
                    WHERE event_id = $1
                """, row['event_id'])
                
            except Exception as e:
                # Increment retry, will be picked up later
                await conn.execute("""
                    UPDATE outbox_events 
                    SET retry_count = retry_count + 1,
                        last_error = $2
                    WHERE event_id = $1
                """, row['event_id'], str(e))
                logger.error(f"Failed to publish event {row['event_id']}: {e}")
        
        await asyncio.sleep(0.1)  # 100ms polling interval
```

**WAL-Based Relay (PostgreSQL 10+):**
For lower latency without polling, use logical replication to stream outbox changes:

```sql
-- Create publication for outbox only
CREATE PUBLICATION outbox_pub FOR TABLE outbox_events;

-- Relay subscribes via logical replication protocol
-- Receives events immediately on commit, no polling delay
```

### 44.3.4 Ordering and Idempotency Guarantees

**Ordering:**
Outbox preserves event order within an aggregate (same `aggregate_id`) due to serial `event_id` and ordered polling:

```sql
-- Ensure aggregate-level ordering
SELECT * FROM outbox_events 
WHERE published_at IS NULL
ORDER BY aggregate_id, event_id  -- Process in order per aggregate
```

**Idempotency:**
Consumers must handle duplicate deliveries (at-least-once semantics):

```sql
-- Consumer deduplication
INSERT INTO processed_events (event_id, processed_at)
VALUES ('evt-uuid-from-message', now())
ON CONFLICT (event_id) DO NOTHING;

-- If rows affected = 0, duplicate detected, skip processing
```

### 44.3.5 Cleanup Strategies

Outbox tables grow indefinitely without cleanup:

```sql
-- Strategy 1: Delete after publishing (if no audit requirement)
DELETE FROM outbox_events 
WHERE published_at < now() - interval '1 hour';

-- Strategy 2: Archive then delete
INSERT INTO outbox_archive 
SELECT * FROM outbox_events 
WHERE published_at < now() - interval '7 days';

DELETE FROM outbox_events 
WHERE published_at < now() - interval '7 days';

-- Strategy 3: Partitioning with DROP
-- Monthly partitions, drop old partitions after archival to S3
DROP TABLE outbox_events_2024_01;  -- Instant, no vacuum bloat
```

## 44.4 Job Queues (SKIP LOCKED Pattern)

For background work processing (image resizing, email sending, report generation), PostgreSQL implements durable job queues using the `SKIP LOCKED` concurrency feature.

### 44.4.1 Queue Table Design

```sql
-- Job queue table
CREATE TABLE job_queue (
    job_id bigserial PRIMARY KEY,
    queue_name text NOT NULL DEFAULT 'default',
    job_type text NOT NULL,           -- 'send_email', 'resize_image'
    payload jsonb NOT NULL,
    priority int DEFAULT 0,           -- Higher = more important
    attempts int DEFAULT 0,
    max_attempts int DEFAULT 3,
    status job_status DEFAULT 'pending',  -- pending, processing, failed, completed
    scheduled_for timestamptz DEFAULT now(),
    created_at timestamptz DEFAULT now(),
    processed_at timestamptz,
    processed_by text,                -- Worker identifier
    error_message text
);

-- Critical index for efficient dequeuing
CREATE INDEX idx_job_queue_fetch 
ON job_queue (queue_name, scheduled_for, priority DESC, job_id)
WHERE status = 'pending';

-- Partial index for failed job analysis
CREATE INDEX idx_job_queue_failed 
ON job_queue (queue_name, created_at) 
WHERE status = 'failed';
```

**Status Enum:**
```sql
CREATE TYPE job_status AS ENUM ('pending', 'processing', 'failed', 'completed');
```

### 44.4.2 The SKIP LOCKED Pattern

`SELECT ... FOR UPDATE SKIP LOCKED` allows concurrent workers to dequeue without blocking:

```sql
-- Worker dequeuing (competitive consumption)
WITH next_job AS (
    SELECT job_id, job_type, payload, attempts
    FROM job_queue
    WHERE queue_name = 'emails'
      AND status = 'pending'
      AND scheduled_for <= now()
    ORDER BY priority DESC, job_id  -- Fair ordering
    LIMIT 1
    FOR UPDATE SKIP LOCKED  -- Skip jobs locked by other workers
)
UPDATE job_queue 
SET 
    status = 'processing',
    attempts = attempts + 1,
    processed_by = current_setting('application.worker_id', true),
    processed_at = now()
FROM next_job
WHERE job_queue.job_id = next_job.job_id
RETURNING job_queue.*;
-- Returns exactly one job, atomically reserved for this worker
```

**Why SKIP LOCKED Matters:**
Without `SKIP LOCKED`, concurrent workers would block on the first row, creating serialization bottlenecks. `SKIP LOCKED` allows workers to skip rows already locked, grabbing the next available job immediately.

### 44.4.3 Worker Implementation

**Ruby (Sidekiq-style):**
```ruby
class Worker
  def run
    loop do
      job = fetch_job
      break unless job
      
      begin
        process(job)
        complete_job(job)
      rescue => e
        fail_job(job, e)
      end
    end
  end
  
  private
  
  def fetch_job
    # Raw SQL with SKIP LOCKED
    ActiveRecord::Base.connection.execute(<<-SQL).first
      WITH next_job AS (
        SELECT job_id FROM job_queue 
        WHERE status = 'pending' AND scheduled_for <= now()
        ORDER BY priority DESC, job_id
        LIMIT 1
        FOR UPDATE SKIP LOCKED
      )
      UPDATE job_queue 
      SET status = 'processing', attempts = attempts + 1
      FROM next_job
      WHERE job_queue.job_id = next_job.job_id
      RETURNING *
    SQL
  end
  
  def complete_job(job)
    ActiveRecord::Base.connection.execute(
      "DELETE FROM job_queue WHERE job_id = #{job['job_id']}"
      # Or UPDATE SET status = 'completed' if audit trail needed
    )
  end
  
  def fail_job(job, error)
    ActiveRecord::Base.connection.execute(<<-SQL)
      UPDATE job_queue 
      SET 
        status = CASE 
          WHEN attempts >= max_attempts THEN 'failed'::job_status 
          ELSE 'pending'::job_status 
        END,
        error_message = '#{escape_string(error.message)}',
        scheduled_for = now() + (attempts || 1) * interval '5 minutes'  -- Exponential backoff
      WHERE job_id = #{job['job_id']}
    SQL
  end
end
```

### 44.4.4 Priority and Delayed Jobs

**Priority Levels:**
```sql
-- Insert with priority (higher = more urgent)
INSERT INTO job_queue (job_type, payload, priority)
VALUES ('send_email', '{"to": "user@example.com"}', 10);

-- Urgent jobs (priority 100) processed before default (0)
ORDER BY priority DESC, job_id
```

**Delayed Execution:**
```sql
-- Schedule for future processing
INSERT INTO job_queue (job_type, payload, scheduled_for)
VALUES (
    'reminder', 
    '{"user_id": 123}', 
    now() + interval '1 day'
);

-- Worker query includes scheduled_for check
WHERE scheduled_for <= now()
```

### 44.4.5 Dead Letter Queues

Failed jobs exceeding retry limits require separate handling:

```sql
-- Move to dead letter queue (separate table)
WITH failed_job AS (
    UPDATE job_queue 
    SET status = 'failed'
    WHERE job_id = $1 
      AND attempts >= max_attempts
    RETURNING *
)
INSERT INTO dead_letter_queue 
SELECT *, now() as failed_at FROM failed_job;

-- Alert on dead letter queue growth
SELECT count(*) FROM dead_letter_queue 
WHERE failed_at > now() - interval '1 hour';
```

## 44.5 Limitations and External Alternatives

While PostgreSQL queues work for moderate loads, specific thresholds demand dedicated message brokers.

### 44.5.1 PostgreSQL Queue Limitations

**Throughput Ceiling:**
- Practical limit: ~1,000-2,000 jobs/second dequeue rate across all workers
- Beyond this, `SKIP LOCKED` contention and vacuum overhead dominate

**Connection Exhaustion:**
Each worker requires a dedicated connection (long-running transaction during job processing):
```sql
-- If 100 workers processing 10-second jobs
-- Requires 100 concurrent connections
-- At 1000 workers, max_connections exceeded or pool exhausted
```

**Vacuum Bloat:**
High-churn queue tables (millions of jobs/day) generate massive dead tuple bloat:
```sql
-- Queue table with 1M jobs/hour processed
-- Creates 1M dead tuples/hour
-- Requires aggressive autovacuum or constant bloat
```

**No Priority Queueing Efficiency:**
PostgreSQL cannot efficiently implement strict priority queues (skip low-priority jobs to find high-priority). The index scan must traverse all pending jobs.

### 44.5.2 Migration Triggers

Move to Redis/RabbitMQ/SQS when:
- Dequeue rate > 2,000/second sustained
- Job latency requirements < 10ms (PostgreSQL polling minimum ~5-10ms)
- Worker count > 200 (connection pool pressure)
- Complex routing required (topic exchanges, dead letter routing)
- Need for visibility timeout extensions (SQS feature)

**Hybrid Architecture:**
```sql
-- Use PostgreSQL for durability-critical events (Outbox)
-- Use Redis for high-volume, ephemeral jobs (cache warming)

-- Or: Tiered queue
-- Fast path: Redis for non-critical jobs
-- Slow path: PostgreSQL for must-not-lose jobs (payments, webhooks)
```

## 44.6 Operational Considerations

### 44.6.1 Monitoring Queue Health

```sql
-- Queue depth alerting (backlog detection)
SELECT 
    queue_name,
    count(*) FILTER (WHERE status = 'pending') as pending,
    count(*) FILTER (WHERE status = 'processing') as processing,
    count(*) FILTER (WHERE status = 'failed') as failed,
    max(created_at) FILTER (WHERE status = 'pending') as oldest_pending,
    avg(extract(epoch from (now() - created_at))) 
        FILTER (WHERE status = 'pending') as avg_wait_seconds
FROM job_queue
GROUP BY queue_name;

-- Alert if pending > 10000 or oldest_pending > now() - interval '1 hour'
```

### 44.6.2 Vacuum and Partitioning for Queues

High-churn queue tables require special handling:

```sql
-- Aggressive autovacuum for queue tables
ALTER TABLE job_queue SET (
    autovacuum_vacuum_scale_factor = 0,
    autovacuum_vacuum_threshold = 1000,
    autovacuum_vacuum_cost_limit = 2000,
    fillfactor = 50  -- Leave room for HOT updates (status changes)
);

-- Or partition by time for easy rotation
CREATE TABLE job_queue_2024_01 PARTITION OF job_queue 
FOR VALUES FROM ('2024-01-01') TO ('2024-02-01');

-- Drop old completed partitions instead of deleting rows
DROP TABLE job_queue_2023_12;  -- Instant, no vacuum needed
```

### 44.6.3 Connection Pool Integration

Queue workers must use separate connection pools from application servers:

```yaml
# Application config
databases:
  app:
    pool_size: 20          # Web application queries
  queue:
    pool_size: 100         # Dedicated to workers
    max_lifetime: 3600     # Longer-lived connections acceptable
    checkout_timeout: 5    # Fail fast if no connections
```

## Chapter Summary

In this chapter, you learned:

1. **LISTEN/NOTIFY**: Use for ephemeral, real-time signaling (cache invalidation, UI updates) where missed messages are acceptable. Respect the 8KB payload limit and implement complementary polling for reliability. Never use for critical financial events or high-frequency (>100/sec) messaging.

2. **Outbox Pattern**: Implement transactional outbox tables to solve the dual-write problem in microservices. Write events atomically with business data, use a relay process to publish to message buses, and ensure consumers handle at-least-once delivery with idempotency checks. Clean up published events to prevent table bloat.

3. **Job Queues**: Use `SELECT ... FOR UPDATE SKIP LOCKED` for high-performance concurrent job dequeuing. Design queue tables with partial indexes on pending status, implement exponential backoff for failures, and move exhausted jobs to dead letter queues. Respect PostgreSQL's practical limit of ~1,000-2,000 jobs/second dequeue rate.

4. **Limitations**: Migrate to dedicated message brokers (Redis, RabbitMQ, SQS) when dequeue rates exceed 2,000/sec, latency requirements drop below 10ms, or worker counts exceed 200 connections. PostgreSQL queues excel at durability and exactly-once semantics but suffer from vacuum bloat and connection overhead at high scale.

5. **Operational Management**: Monitor queue depth, oldest pending job age, and failed job rates. Configure aggressive autovacuum for queue tables or use time-based partitioning for easy data rotation. Maintain separate connection pools for queue workers to prevent application connection starvation.

---

**Next:** In Chapter 45, we will explore Testing Strategies—covering unit tests for database functions, integration tests with ephemeral databases, property-based testing for SQL correctness, and deterministic test data management that ensures reproducible results across environments.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='43. geospatial.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../12. Testing_delivery_and_team_practices/45. local_development_workflows.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
